In [ ]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR
from src.data import split_data, prepare_feature_store_data_for_training

In [98]:
data = pd.read_csv(TRANSFORMED_DATA_DIR / "ts_tabular_2022_10_to_2024_11.csv")
data.head()

,datetime,demand,ba_code
0,2022-10-01,51628,AECI
1,2022-10-02,53127,AECI
2,2022-10-03,54708,AECI
3,2022-10-04,53345,AECI
4,2022-10-05,53356,AECI


In [99]:
data = prepare_feature_store_data_for_training(data)
data.head()

,ba_AECI,ba_AVA,ba_AZPS,ba_BANC,ba_BPAT,ba_CHPD,ba_CISO,ba_CPLE,ba_CPLW,ba_DOPD,...,ba_SWPP,ba_TAL,ba_TEC,ba_TEPC,ba_TIDC,ba_TPWR,ba_TVA,ba_WACM,ba_WALC,ba_WAUW
datetime,,,,,,,,,,,,,,,,,,,,,
2022-10-01,51628.0,27070.0,96193.0,46398.0,123905.0,3745.0,592567.0,123640.0,11797.0,4617.0,...,649266.0,6601.0,49325.0,34960.0,8222.0,10804.0,348678.0,80299.0,24633.0,1820.0
2022-10-02,53127.0,28039.0,97208.0,42814.0,125317.0,3655.0,560074.0,124418.0,11759.0,4583.0,...,647709.0,6682.0,51702.0,37336.0,7194.0,10773.0,345900.0,79702.0,26100.0,1749.0
2022-10-03,54708.0,30110.0,96570.0,47041.0,133353.0,3790.0,623658.0,132803.0,12155.0,4732.0,...,689771.0,6886.0,54270.0,35662.0,8346.0,11228.0,373596.0,80536.0,25310.0,1915.0
2022-10-04,53345.0,30764.0,88963.0,48332.0,134664.0,3831.0,654561.0,134430.0,12294.0,4829.0,...,687579.0,6781.0,53101.0,31415.0,8765.0,10927.0,378071.0,80522.0,23938.0,1838.0
2022-10-05,53356.0,30421.0,91984.0,50362.0,135464.0,3815.0,664304.0,136348.0,12482.0,4854.0,...,687179.0,7009.0,54212.0,30993.0,8457.0,10962.0,376763.0,81362.0,23839.0,1828.0


In [100]:
import numpy as np

def forwardfill_missing_values(data: pd.DataFrame) -> pd.DataFrame:
    df = data.copy()
    df = df.replace(-1, np.nan).ffill()
    return df

data = forwardfill_missing_values(data)

### Build benchmark models

Build several benchmark models to compare model performance against. 

- Naive: previous day's demand as forecast. 
- Lag 7: same day a week ago as forecast. 
- Rolling mean 7: rolling weekly average as forecast. 
- Lag 30: use same day one month ago as forecast. 
- Rolling mean 30: rolling monthly average as forecast. 
- Lag 365: use same day one year ago as forecast. 
- Rolling mean 365: rolling annual average as forecast. 

In [138]:
benchmark = data.copy()
target_columns = benchmark.filter(like="ba_").columns

In [ ]:
def get_lag_columns(data: pd.DataFrame, lag: int, target_columns: list[str] = target_columns) -> pd.DataFrame:
    
    df = data.copy()

    lag_columns = {
        f"lag{lag}_{col}".replace("ba_", ""): df[col].shift(periods=lag)
        for col in target_columns
    }

    # Convert the dictionary to a DataFrame
    lag_df = pd.DataFrame(lag_columns)

    # Concatenate the original and new DataFrame along the column axis
    df = pd.concat([df, lag_df], axis=1)

    return df

benchmark = get_lag_columns(benchmark, lag=1, target_columns=target_columns)
benchmark = get_lag_columns(benchmark, lag=7, target_columns=target_columns)
benchmark = get_lag_columns(benchmark, lag=30, target_columns=target_columns)
benchmark = get_lag_columns(benchmark, lag=365, target_columns=target_columns)

In [ ]:
def get_rolling_average_columns(data: pd.DataFrame, window: int, target_columns: list[str] = target_columns) -> pd.DataFrame:

    df = data.copy()

    for col in target_columns:
        result = (
            df[col]
            .rolling(window=window)  
            .agg(["mean"])  
            .shift(freq="1D")
        )

        result.columns = [f"rolling{window}_{col}".replace("ba_", "")]

        df = df.merge(result, how="left", left_index=True, right_index=True)

        # df.dropna(inplace=True)

    return df

benchmark = get_rolling_average_columns(benchmark, window=7, target_columns=target_columns)
benchmark = get_rolling_average_columns(benchmark, window=30, target_columns=target_columns)
benchmark = get_rolling_average_columns(benchmark, window=365, target_columns=target_columns)

In [141]:
_, data_test = split_data(benchmark, end_train="2024-08-31 23:59:00")

data_test.head()

data_train.shape=(701, 424)
data_test.shape=(90, 424)
Train dates : 2022-10-01 00:00:00 --- 2024-08-31 00:00:00   (n=701)
Test dates  : 2024-09-01 00:00:00 --- 2024-11-29 00:00:00   (n=90)


,ba_AECI,ba_AVA,ba_AZPS,ba_BANC,ba_BPAT,ba_CHPD,ba_CISO,ba_CPLE,ba_CPLW,ba_DOPD,...,rolling365_SWPP,rolling365_TAL,rolling365_TEC,rolling365_TEPC,rolling365_TIDC,rolling365_TPWR,rolling365_TVA,rolling365_WACM,rolling365_WALC,rolling365_WAUW
datetime,,,,,,,,,,,,,,,,,,,,,
2024-09-01,62664.0,33645.0,124723.0,52270.0,145673.0,4388.0,671602.0,191008.0,12108.0,6262.0,...,784763.161644,7703.427397,60717.526027,39071.487671,7946.528767,12513.783562,449329.728767,91706.753425,22849.778082,2230.605479
2024-09-02,54850.0,32264.0,125815.0,50889.0,143435.0,4390.0,690107.0,178152.0,12806.0,6303.0,...,784590.989041,7707.887671,60716.997260,39074.172603,7949.082192,12512.284932,449329.824658,91694.019178,22854.268493,2229.950685
2024-09-03,51463.0,34463.0,132812.0,57395.0,149742.0,4664.0,772327.0,163165.0,12463.0,6608.0,...,784292.800000,7712.800000,60721.575342,39073.353425,7953.665753,12511.041096,449331.369863,91702.027397,22857.169863,2229.701370
2024-09-04,57470.0,35155.0,136295.0,65881.0,154963.0,4614.0,829458.0,164773.0,11507.0,6566.0,...,783992.235616,7717.698630,60731.241096,39077.893151,7959.989041,12512.556164,449274.049315,91726.980822,22866.273973,2230.019178
2024-09-05,62148.0,36023.0,142283.0,67085.0,162581.0,4705.0,872459.0,163917.0,11801.0,6566.0,...,783680.120548,7717.901370,60725.846575,39086.071233,7964.958904,12513.561644,449182.076712,91750.646575,22880.071233,2229.904110


We use the mean absolute error averaged over all series to evaluate each model. To do this, we need to filter for the model type, e.g. 'lag7', find the MAE for this type with respect to each series, then average the results. 

In [142]:
from sklearn.metrics import mean_absolute_error
import numpy as np

all_means = []
for col in target_columns:
    # Getting the demand and benchmark scores for a given BA over the test period
    ba_string = col.replace("ba_", "")
    tmp = benchmark.filter(regex=f"{ba_string}$").loc[data_test.index].copy()
    ba = tmp.iloc[:, 0]
    maes = []
    # Calculating the MAE by comparing each score against the demand
    for c in tmp.columns[1:]:
        bm = tmp[c].copy()
        mae = mean_absolute_error(ba, bm)
        maes.append(mae)

    all_means.append(np.array(maes))

# Averaging each score over all of the BAs
stacked = np.stack(all_means)
final_means = np.mean(stacked, axis=0)

In [143]:
benchmark_names = [col.replace("_SC", "") for col in benchmark.filter(regex="SC$").columns[1:]]
pd.DataFrame({benchmark_names[i]: [final_means[i]] for i in range(len(benchmark_names))})

,lag1,lag7,lag30,lag365,rolling7,rolling30,rolling365
0,7588.685535,15179.846541,26397.836688,16671.280294,10999.030219,15220.508029,20951.642148


On this basis, using the previous day's demand as the forecast gives the smallest MAE, so we'll use this as a benchmark for comparison moving forward. 

In [ ]:
from typing import Tuple

def make_baseline_predictions(test_data: pd.DataFrame) -> Tuple[pd.DataFrame, float, list[float]]:
    """
    Wrapper to return a batch of predictions using the best baseline model to assess
    performance.
    
    Args: 
        test_data: data that we are using for evaluation
        
    Returns:
        Tuple containing the predictions, the average MAE over all BAs, and the individual MAE 
        for each BA.
    """
    predictions = get_lag_columns(
        data=test_data, lag=1, target_columns=test_data.filter(like="ba_").columns
    )
    
    predictions.dropna(inplace=True)
    
    num_bas = data_test.filter(like="ba_").shape[1]

    maes = [mean_absolute_error(predictions.iloc[:, i], predictions.iloc[:, i + num_bas]) for i in range(num_bas)]
    average_mae = np.mean(maes)
    
    return predictions, average_mae, maes

In [ ]:
benchmark = data.copy()
# Take one day extra because we need the lag for 01/09
_, data_test = split_data(benchmark, end_train="2024-08-30 23:59:00")
predictions, average_mae, _ = make_baseline_predictions(data_test)
predictions.head()

data_train.shape=(700, 53)
data_test.shape=(91, 53)
Train dates : 2022-10-01 00:00:00 --- 2024-08-30 00:00:00   (n=700)
Test dates  : 2024-08-31 00:00:00 --- 2024-11-29 00:00:00   (n=91)


,ba_AECI,ba_AVA,ba_AZPS,ba_BANC,ba_BPAT,ba_CHPD,ba_CISO,ba_CPLE,ba_CPLW,ba_DOPD,...,lag1_SWPP,lag1_TAL,lag1_TEC,lag1_TEPC,lag1_TIDC,lag1_TPWR,lag1_TVA,lag1_WACM,lag1_WALC,lag1_WAUW
datetime,,,,,,,,,,,,,,,,,,,,,
2024-09-01,62664.0,33645.0,124723.0,52270.0,145673.0,4388.0,671602.0,191008.0,12108.0,6262.0,...,836525.0,9667.0,72376.0,47572.0,9612.0,10701.0,503558.0,96457.0,26352.0,2076.0
2024-09-02,54850.0,32264.0,125815.0,50889.0,143435.0,4390.0,690107.0,178152.0,12806.0,6303.0,...,819633.0,9674.0,70856.0,46435.0,8843.0,10555.0,459868.0,94341.0,25540.0,2112.0
2024-09-03,51463.0,34463.0,132812.0,57395.0,149742.0,4664.0,772327.0,163165.0,12463.0,6608.0,...,788292.0,10292.0,68896.0,48307.0,8863.0,10100.0,463039.0,98950.0,25795.0,2222.0
2024-09-04,57470.0,35155.0,136295.0,65881.0,154963.0,4614.0,829458.0,164773.0,11507.0,6566.0,...,824224.0,10513.0,71982.0,49042.0,10029.0,10750.0,472635.0,102235.0,27279.0,2272.0
2024-09-05,62148.0,36023.0,142283.0,67085.0,162581.0,4705.0,872459.0,163917.0,11801.0,6566.0,...,867693.0,9294.0,71154.0,49524.0,11123.0,11279.0,473743.0,99052.0,28861.0,2068.0


In [161]:
average_mae

7588.685534591197